In [1]:
import pandas as pd
import numpy as np
from numpy import nan

#store as a dataframe
pd.set_option("display.max_colwidth", -1)
df = pd.read_excel("Detail Trip Report 19-02-2021.xls",skiprows =15)
df1 = pd.read_excel("Detail Trip Report 29-02-2020.xls",skiprows =15)
df2 = pd.read_excel("Detail Trip Report 30-04-2020.xls",skiprows =15)
df3 = pd.read_excel("Detail Trip Report 30-06-2020.xls",skiprows =15)
df4 = pd.read_excel("Detail Trip Report 30-09-2020.xls",skiprows =15)
df5 = pd.read_excel("Detail Trip Report 30-11-2020.xls",skiprows =15)
df6 = pd.read_excel("Detail Trip Report 31-01-2021.xls",skiprows =15)
df7 = pd.read_excel("Detail Trip Report 31-03-2020.xls",skiprows =15)
df8 = pd.read_excel("Detail Trip Report 31-05-2020.xls",skiprows =15)
df9 = pd.read_excel("Detail Trip Report 31-07-2020.xls",skiprows =15)
df10 = pd.read_excel("Detail Trip Report 31-08-2020.xls",skiprows =15)
df11 = pd.read_excel("Detail Trip Report 31-10-2020.xls",skiprows =15)
df12 = pd.read_excel("Detail Trip Report 31-12-2020.xls",skiprows =15)
frames = [df, df1, df2, df3,df4,df5,df6,df7,df8,df9,df10,df11,df12]
combined = pd.concat(frames,sort=False)
original = combined
combined.head(10)


C:\Users\yjkwak.2018\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


FileNotFoundError: [Errno 2] No such file or directory: 'Detail Trip Report 19-02-2021.xls'

In [ ]:
combined.to_csv('Combined_Detail_Trip_Report.csv',index=False)

In [ ]:
combined.head(10)

In [ ]:
combined =combined.reset_index().drop('index', axis=1)
combined.info()

In [ ]:
ef = combined
ef =  ef[['Case','Date','Long','Lat','Odometer','Location','Speed']]
ef =  ef[ef['Date'].notna()]
ef.head(10)

In [ ]:
ff = ef[(ef['Date'] != "Driver")]
ff = ff[ff['Date'] != "Date"]
ff.head(20)

In [ ]:
print(ff.iloc[37050:37090])

In [ ]:
ff =ff.reset_index().drop('index', axis=1)
ff.info()

In [ ]:
for ind, row in ff.iterrows():
    print(ind, end="\r")
    if str(row['Location']) == 'nan':
        add_index = ind
    if len(str(row['Location'])) < 10 and str(row['Location']) != 'nan':
        ff.at[add_index, 'Location'] = row['Location']
ff['Trip'] = ""
ff.head(10)

In [ ]:
gf = ff
count = 1
archive = ""

for ind, row in gf.iterrows():
    print(ind, end="\r")
    
    if str(row['Case']) == "# Trip":
        count +=1
        if str(row['Location']) not in ["YP5064C", "YP5475A", "YP5606P", "YQ1110Z" ]:
            pass 
        else:
            archive = str(row['Location'])
            
    gf.at[ind, 'Trip'] = count
    
    
    if str(row['Location']) not in ["YP5064C", "YP5475A", "YP5606P", "YQ1110Z" ]:
            gf.at[ind, 'Location'] = archive
gf.head(10)

In [ ]:
gf = gf[gf['Date'] != "Unavailable"]
gf = gf.reset_index().drop('index', axis=1)
gf.columns = ['Case','Date','Long','Lat','Odometer','Registration','Speed','Trip']
gf

In [ ]:
for ind, row in gf[gf['Registration'] == ""].iterrows():
    print(ind, end="\r")
    gf.at[ind, 'Registration'] = "YP5064C"
gf

In [ ]:
gf.groupby('Registration')['Registration'].count()

In [ ]:
gf.groupby('Case')['Case'].count()

In [ ]:
gf.info()

In [ ]:
gf.to_csv('Detail_Trip_Report_final.csv',index=False)

In [ ]:
gaf = pd.read_csv("Detail_Trip_Report_final.csv",skiprows =15)